# Functions for training deep narrow ReLU neural networks with Lasso

- neural nets can have depth of L = 2,3,4 layers
- run ```analyze_cvx_nn()``` with arguments $L$ specifying the depth and $dataset$ specifying one of three example data sets in our paper

In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import math
plt.rcParams.update({'font.size': 20})
import warnings
warnings.filterwarnings('ignore')

from fractions import Fraction

(CVXPY) Mar 18 01:00:47 PM: Encountered unexpected exception importing solver SCS:
ImportError('dlopen(/Users/emi/Library/Python/3.7/lib/python/site-packages/_scs_direct.cpython-37m-darwin.so, 2): Symbol not found: _aligned_alloc\n  Referenced from: /Users/emi/Library/Python/3.7/lib/python/site-packages/scs/.dylibs/libgomp.1.dylib (which was built for Mac OS X 10.15)\n  Expected in: /usr/lib/libSystem.B.dylib\n in /Users/emi/Library/Python/3.7/lib/python/site-packages/scs/.dylibs/libgomp.1.dylib')


### Functions to solve general Lasso problem for deep narrow ReLU networks

- uses the tuple $i=(s,j,k)$ to index dictionary columns (Lasso features) as defined in our paper (Section 3.1). The functions ```convertTuple2index()```  and ```convertIndex2Tuple()``` convert between i and (s,j,k).

- recursive definition of Lasso features is used (Appendix C) instead of equivalent functional definition (Section 3.1). This is to make the code simple and verify the equivalences of the two definitions. The function ```reludic()``` computes the $(s,j,k)^{th}$ Lasso feature at a point $x$.

- The function ```lasso_nn_deepnarrow()``` solves the Lasso problem with cvxpy

In [2]:
def relu(x):
    return np.maximum(0,x)

def fixindex(index):
    return index-1

def s2index(s):
    #s is array with s[i] is -1 or 1
    arraylen = len(s)
    ss = np.zeros((arraylen,))
    for i in range(arraylen):
        ss[i]=0.5*(s[i]+1) #0 or 1
    return ss

def sindex2s(s):
    #s is array with s[i] is 0 or 2
    arraylen = len(s)
    ss = np.zeros((arraylen,))
    for i in range(arraylen):
        ss[i]=2*s[i]-1 #-1 or 1
    return ss   

def undofixindex(index):
    return index+1

def reflect(a1, a2):
    return 2*a2-a1

def reludic(l,s,j,k,data,x): 
    #x is a scalar. k is a scalar. data is the sorted data matrix X. s and j are arrays of length l-1. l is a scalar: 2,3 or 4
    if l==1:
        return x
    
    l = l-1 
    if l==1 and k==1:
        a=reflect(data[j[fixindex(1)]],data[j[fixindex(2)]])[0]
    else:
        a = data[j[fixindex(l)]][0] #[0] for the way makedata outputs data
            
    xterm = reludic(l,s,j,k,data,x)
    aterm = reludic(l,s,j,k,data,a)
    sterm = s[fixindex(l)]
          
    out = relu(sterm*(xterm-aterm))
    
    return out

def convertTuple2index(s,j,k,N):
    #s is array of -1, 1. j is array of 0,...,N-1's. k is 0 or 1. 
    index=0
    arraylen = len(s) #=length(j)
    ss = s2index(s)
    
    #add j
    for i in range(arraylen):
        index = index + j[i]*(N**i)
        
    #add s
    for i in range(arraylen):
        index = index + ss[i]*(2**i)*(N**arraylen) 
        
    #add k
    index = index + k*(N**arraylen)*(2**arraylen)
    
    #zero-index
    return index

def convertIndex2Tuple(index,N,arraylen):
    #arraylen is L-1, so 1,2,or 3
    
    s = np.zeros((arraylen,))
    j = np.zeros((arraylen,))
    
    #get k
    q, mod = divmod(index, (N**arraylen)*(2**arraylen))
    k=q
    
    #get s
    for i in range(arraylen):
        q, mod =divmod(mod, (N**arraylen)*(2**(arraylen-i-1)))
        s[arraylen-i-1]=q   
     
    #get j
    for i in range(arraylen):
        q, mod = divmod(mod, (N**(arraylen-i-1)))
        j[arraylen-i-1]=q
            
    s = sindex2s(s) 
    s = s.astype(int)
    j = j.astype(int)
    
    return s, j, k

def lasso_nn_deepnarrow(L, X_train, y_train, X_test, findoptset=False, optval=[], β = 1e-7, activation_func = relu, verbose=False):
    activation_func = relu
    num_samples = len(X_train)
    num_bidual_var = (2**(L-1))*(num_samples**(L-1))
    if L==4:
        num_bidual_var=2*num_bidual_var #k=1 are the highest indices 
    
    #define variables z and ξ and lasso cost for bidual
    z=cp.Variable((num_bidual_var,1)) #first argument dimension d=1
    ξ=cp.Variable((1,1)) #bias term
    
    A=np.zeros((num_samples,num_bidual_var))
    arraylen = L-1
    for i in range(num_bidual_var):
        s,j,k=convertIndex2Tuple(index=i,N=num_samples,arraylen=arraylen)
        A[:,i]=np.squeeze(reludic(l=L,s=s,j=j,k=k,data=X_train,x=X_train))
        
    cost = 0.5*cp.sum_squares(y_train-A@z-ξ*np.ones((num_samples,1)))+β*cp.norm(z,1)
    constraints = []
    
    if findoptset:
        constraints = [cost<=optval]
        cost = cp.sum_squares(z)
        
    
    #set tolerances as best as possible
    params = {
        #"MSK_IPAR_NUM_THREADS": 8,
        #"MSK_IPAR_INTPNT_MAX_ITERATIONS": 100,
        #"MSK_IPAR_OPTIMIZER": 0, # auto 0, interior point 1, conic 2
        "MSK_DPAR_INTPNT_CO_TOL_REL_GAP": 1e-12,
        #"MSK_DPAR_INTPNT_TOL_PSAFE": 1e-8,
        #"MSK_IPAR_OPTIMIZER": "free",
        #"MSK_IPAR_INTPNT_SOLVE_FORM": 1
      }
   
    #solve bidual problem 
    prob=cp.Problem(cp.Minimize(cost),constraints)
    
    prob.solve(solver=cp.MOSEK,warm_start=True,verbose=verbose,mosek_params=params)
    cvx_opt=prob.value
    mstar = np.sum(np.abs(z.value)>0) #number of neurons
   
    if prob.status != "optimal":
        print("Convex: Status convex: ",prob.status)
    if verbose:
        print("2-layer convex program objective value: ",cvx_opt)
        print("opt bidual var z = ", np.round(z.value, 3))
        print("opt bidual bias var xi = ", ξ.value)
        print("number of neurons (m*): ", mstar)
        
    return A, z.value, ξ.value, cvx_opt


def makedata(X_train, y_train, include_train_in_test=False):
    #sort X_train in descending order
    #reorder labels y in the same order
    #make testing data to have similar range as X_train
    
    sort_indices = np.argsort(-1*X_train) #descend!
    X_train = X_train[sort_indices] #sort in descending order (for makeA())
    X_train = np.expand_dims(X_train, axis=1) #reshape to n x 1 vector
    num_samples_train = len(X_train)

    y_train = y_train[sort_indices] #sort y in the same order as x
    y_train = np.expand_dims(y_train, axis=1)
    X_test = np.linspace(np.min(X_train)-1,np.max(X_train)+1,100)
    if include_train_in_test:
        X_test = np.hstack((X_test, X_train.ravel())) #include train in test data
    X_test = np.expand_dims(X_test, axis=1)
    
    return X_train, y_train, X_test

### Functions to generate and plot theoretical solutions to the min norm version of Lasso for examples given in our paper

- used when the $dataset$ argument in ```analyze_cvx_nn()``` is set to "opt"

In [3]:
def ramp(a,b,x):
    if x <= a:
        return 0
    if x<= b:
        return x-a
    
    return b-a

def rampvec(a,b,x):
    out = np.zeros_like(x)
    for i in range(len(x)):
        out[i]=ramp(a=a,b=b,x=x[i])
    
    return out

def get_features(L,x):
    if L==4:
        features = [rampvec(5,8,x)]
        coefs = [1]
    if L==3:
        features = [rampvec(5,7,x),rampvec(5,10,x)]
        coefs = [2/3,1/3]
    if L==2:
        features = [rampvec(5,np.inf,x),rampvec(7,np.inf,x),rampvec(10,np.inf,x)]
        coefs = [1,-2/3,-1/3]
        
    return features, coefs

def datafuncvec(x,L,verbose=False):
    feats, coefs = get_features(L=L,x=x)
    out = np.zeros_like(feats[0])
    for i, feat in enumerate(feats):
        if verbose:
            print('feat=',feat,'coef=',coefs[i],'out=',out)
        out = out + coefs[i]*feat

    return out



### Function called to produce figures in our paper

In [4]:
def analyze_cvx_nn(L, dataset, save, figtitle, labelyaxis=False):
    #dataset can be 'zshape', 'bounce', or 'opt'. It specifies the training samples X_train and labels y_train, and plotting parameters.
    
    #plotting constants
    cvxtitle = 'training with Lasso'
    linewidth=3.0
    markersize=100
    width=6
    height=3
    datalabel='$(x_n,y_n)$'
    nnlabel = 'net'

    #get setup
    if dataset == 'zshape':
        X_train = np.array((0,2,6,7)) #training samples. 
        y_train = np.array((0,0,3,3)) #labels
        yaxislim = [min(y_train)-0.2,max(y_train)+0.2]
        xaxislim = [min(X_train)-1,max(X_train)+1]
        xaxis = np.linspace(min(xaxislim)+1,max(xaxislim)-1,int(max(xaxislim)-min(xaxislim)-1))
        plottheory=False
        xlimfeat = [-1,8.5]
        ylimfeat = [-0.5,6.5]
        xaxisfeat = np.arange(9)
        β=1e-8
        
    elif dataset == 'bounce':
        X_train = np.array((0,2,6,7)) 
        y_train = np.array((0,0,3,7))
        yaxislim = [-5.2,12]
        xaxislim = [-1,8.5]
        xaxis = np.arange(9)
        plottheory=False
        xlimfeat = [-1,8.5]
        ylimfeat = [-0.5,6.5]
        xaxisfeat = np.arange(9)
        β=1e-8
        
    elif dataset == 'opt':
        X_train = np.array((0,4,5,7,10,11)) 
        y_train = np.array((0,0,0,2,3,3))
        xaxislim = [-1.5,14]
        yaxislim = [-0.5,5.5]
        xaxis = np.linspace(min(xaxislim)+1,max(xaxislim)-1,int(max(xaxislim)-min(xaxislim)-1))
        plottheory=True #for this example in the paper, we plot the theoretical solution to the min norm problem
        xlimfeat = xaxislim
        ylimfeat = yaxislim
        xaxisfeat = np.array((0,2,4,6,8,10,12))
        β=1e-7
    
    #train
    num_samples = len(X_train)
    X_train, y_train, X_test = makedata(X_train, y_train)
    if not plottheory:
        A, z, ξ, obj=lasso_nn_deepnarrow(L=L, X_train=X_train, y_train=y_train, X_test=X_test, β = β) #needed β=1e-8 or less
        print('optimal objective = ', obj)

    #get nn predictions
    numtestpts = 100
    if dataset == 'opt':
         x= np.linspace(start=min(xaxislim)+1,stop=max(xaxislim)-1,num=numtestpts).reshape((numtestpts,1))
    else:
        x=np.linspace(min(X_train)-0.5,max(X_train)+0.5,numtestpts).reshape((numtestpts,1))
   
    
    if not plottheory:
        test_val = ξ*np.ones((numtestpts,1))
        for i in range(len(A[0,:])):
            s,j,k=convertIndex2Tuple(index=i,N=num_samples,arraylen=L-1)
            test_val = test_val + z[i]*(reludic(l=L,s=s,j=j,k=k,data=X_train,x=x))
    else:
        test_val = datafuncvec(x=x,L=L)   

    #plot predictions
    plt.figure(figsize=(width,height))
    plt.scatter(X_train, y_train, label = datalabel, marker="o", color="red", s=markersize)
    plt.plot(x,test_val, label = nnlabel,  linewidth=linewidth, color="blue")
    plt.xlim(xaxislim)
    plt.ylim(yaxislim)
    plt.legend()
    if (L==2) or (L==3 and dataset=='zshape'):
        plt.title(cvxtitle)
        plt.xticks([])
    elif L<4 and dataset == 'opt':
        plt.xticks([])
    else:
        ax = plt.gca()
        ax.set_xticks(xaxis, labels=xaxis.astype(int))
    if not labelyaxis:
        plt.yticks([])
    if dataset=='bounce':
        plt.ylabel(str(L)+" layers")
    if dataset=='opt' and L==4:
        ax = plt.gca()
        ax.set_xticks(xaxisfeat, labels=xaxisfeat.astype(int))

        
    if save:
        plt.savefig(figtitle + "_pred.pdf", format="pdf", bbox_inches="tight")
    
    if dataset=='zshape':
        return

    #get nonzero features and plot
    plt.figure(figsize=(width,height))
    if L==2:
        plt.title('selected features')
    numfeature=0

    if plottheory:
        features, coefs = get_features(L,x)
        for i, feat in enumerate(features):
            coefstr = str(Fraction(coefs[i]).limit_denominator(10))
            plt.plot(x,feat,label='$z_{i_' + str(i+1) + '}^*='+ coefstr + '$',  linewidth=linewidth)
    else:
        z=np.squeeze(z)
        nonzero_indices = np.nonzero(z)
        for i in nonzero_indices[0]:
            if np.abs(z[i])>0.05:
                s,j,k=convertIndex2Tuple(index=i,N=num_samples,arraylen=L-1)
                print('s,j,k=',s,j,k)
                out = reludic(l=L,s=s,j=j,k=k,data=X_train,x=x)
                if numfeature==0:
                    col = "yellow"
                else:
                    col="green"
                plt.plot(x,out,label='$z_{i_' + str(numfeature+1) + '}^*='+str(round(z[i],2)) + '$', color=col,  linewidth=linewidth)
                numfeature=numfeature+1

    plt.xlim(xlimfeat)
    plt.ylim(ylimfeat)
    if L<4:
        plt.xticks([])
    else:
        ax = plt.gca()
        ax.set_xticks(xaxisfeat, labels=xaxisfeat)
    if dataset == 'opt':
        plt.yticks([])
    
    plt.legend()
    
    if save:
        plt.savefig(figtitle + "_feat.pdf", format="pdf", bbox_inches="tight")


In [5]:
#analyze_cvx_nn(L=3, dataset='opt')